<h3>Introduction to the pyvista tv and imred modules

<i> This is an older notebook and may not be fully up-to-date

Start with some standard imports. We will want to use an "asynchonous" display tool that will remain open, hence the %matplotlib qt<br>
Note that it will be convenient to configure your desktop so that the window focus follows the mouse so you don't have to keep clicking back and forth between browser and display window.

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import astropy.units as u
from pyvista import imred, tv
from pyvista.dataclass import Data

In [9]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
#%matplotlib qt
#t=tv.TV()
#plotinter=True

# following lines only for fully noninteractive demo of notebook, to allow
#    inline demo of plots (but not image display)
%matplotlib inline
t=None
plotinter=False     # normally, want this to be True 

We will start with the tv and imred packages. Load them, and instantiate a display tool in the variable t

Instantiate a pyvista Reducer object. This sets up some basic information for an instrument (e.g., gain, readout noise, overscan type and region) and, optionally, a default directory to read images from. It also defines a format string for the instrument that, if possible, allows files to be identified by a sequence number only.

In [10]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/red/UT211030/kosmos')

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/holtz/red/UT211030/kosmos/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [ 5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


The Reducer object has several methods. <br>
<ul>
<li>The simplest is rd(), which will read from a file into a variable. The returned variable is a CCDData object with attributes data, uncertainty, and mask. The uncertainty is calculated upon reading the image using the noise model as given by the gain and readout noise for the detector. With rd(), as opposed to reduce(), however, the file is simply read as is, without any overscan subtraction, so the uncertainty will not be valid.<br>
<li>The reduce() method will read the file, subtract the overscan, and calculate the uncertainty from the noise model. As we will see below, additional optional keywords to reduce() will perform additional reduction steps.
</ul>

In either case, the file to read can be specfied as either a file number, in which case the routine attempts to associate a file name with the number using the format string for the instrument to create a search string, or by the character file name itself. If a filename is specified, it will be read from the default input directory unless the filename includes a '/', in which case it will be read from the full path as specified.

In [11]:
# all three of these do the same thing. Working with file numbers is convenient!
raw=red.rd(10)
raw=red.rd('BD28d4211.0010.fits')
raw=red.rd('/home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits')

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits


Displaying the image will show that the bias has not been subtracted: move the mouse around in the image to see the image values.

In [13]:
if t is not None :
    t.tv(raw)

Note some useful features in the display tool:
<ul>
    <li> left click: zooms in
    <li> right click: zooms out
    <li> r : restores initial zoom
    <li> - : cycles back to previous image in rolling stack (of 4)
    <li> + : cycles forward in rolling image stack
</ul>
You can specify the display scaling parameters in the tv command with min=low and max=high keywords. You can also drag the colorbar from either side to increase the constrast with the current display scaling parameters (r key in color bar resets the color map).

OK, now read in the image using reduce() to do the overscan subtraction, and display

In [14]:
a=red.reduce(10)
if t is not None :
    t.tv(a)

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


You should now see unilluminated regions with values near zero. 

If you give reduce() the optional display= keyword with a TV instance, then it will show you more information about the overscan subtraction: the overscan box used, and a plot of the overscan region(s). If the detector has multiple amplifiers that have been defined in the configuration, the separate overscan regions will be shown with boxes of different colors, and the region of the chip to which they are applied are shown with boxes of the same color (dashed lines)<br>
With the display= keyword, you will be required to hit a key in the display window to continue: this will be used below as you scan through a number of images to allow you to look at each one.

In [15]:
a=red.reduce(10,display=t)

  Reading file: /home/holtz/red/UT211030/kosmos/BD28d4211.0010.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


You can make calibration products by combining multiple calibration frames with additional Reducer methods:
<ul>
    <li> mkbias(list) : combines images in image list to make bias
    <li> mkdark(list) : combines images to make dark
    <li> mkflat(list) : combines images to make flat
</ul>
The default combination method is to median combine, but you can also specify type='mean' or type='reject' (for sigma clipping, default is 5-sigma, but can be modified with the sigreject= keyword)<br>
If you supply the optional display= keyword, the combined image will be displayed, and then the difference (ratio in the case of a flat) between each input image and the combined image will be shown for your inspection: you will need to hit a key in the display window after each one to continue to the next. Upper right plot will show the histogram of the combined frame, lower right plots will show the histogram of the individual frame differences with the combined frame.

In [16]:
#several different ways of constructing image list:
imlist=range(23,32)    # all frames from 23-31
imlist=[23,24,25,26,27,28,29,30,31]  # explicit list of frame numbers
imlist=sorted(glob.glob(red.dir+'/*bias*'))   # find all frames with bias in name

bias=red.mkbias(imlist,display=t)

  Reading file: /home/holtz/red/UT211030/kosmos/bias.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0024.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0025.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0026.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0027.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0028.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0029.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/bias.0030.fits
  subtracting overscan vector 
  subtracting overscan vector 


You can save the calibration products (which are also CCDData objects) using their write() method. You can then read these back in for subsequent use so you don't have to create them every time you want to use them; use the CCDData (imported above) read() method (unfortunately, you have a specify a unit with this).

In [19]:
#save the product
bias.write('bias.fits',overwrite=True)
#example of using a previously saved object:
bias=Data.read('bias.fits',unit=u.dimensionless_unscaled)

appending uncertainty
appending bitmask
INFO: first HDU with data is extension 1. [pyvista.dataclass]


the mkflat() method works similarly. For flats, the images are normalized before combining, using the region specified in the instrument configuration file. <br>
There is an optional spec= keyword, which, if set to true, will remove the shape in the spectral dimension for a spectroscopic instrument (note that the spectral dimension is along columns unless transpose is specified in the configuration file)

In [20]:
flat=red.mkflat(range(18,23),display=t,spec=False)
sflat=red.mkflat(range(18,23),display=t,spec=True)
if t is not None :
    t.tv(flat,min=0.5,max=1.5)
    t.tv(sflat,min=0.8,max=1.2)

  Reading file: /home/holtz/red/UT211030/kosmos/flat.0018.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0019.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0018.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0019.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211030/kosmos/flat.0020.fits

Use the +/- keys to switch back and forth to see the two displayed flats.

<h3>ARCTIC darks


Here's an example of using the mkdark() method to make some ARCTIC darks.

In [21]:
ared=imred.Reducer('ARCTIC',conf='quad',dir='/home/holtz/red/UT211031/ARCTIC')
dark=ared.mkdark(range(110,120))

INSTRUMENT: ARCTIC   config: quad
  will use format:  /home/holtz/red/UT211031/ARCTIC/*{:04d}.f*.fits*
         gain:  [2.0]    rn: [ 3.7]
         scale:  0.228   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  1026    26    20   981 
  1026    26  1050   979 
  1052    24    20   981 
  1052    24  1050   979 
  Trim box: 
    SC    NC    SR    NR
     2  1024     0  1024 
     2  1024  1026  1024 
  1076  1024     0  1024 
  1076  1024  1026  1024 
  Norm box: 
    SC    NC    SR    NR
   800   201   800   201 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0110.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0111.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0112.fits
  subtracting overscan vec

For darks, there is an optional keyword clip=$n$, which will set to zero those pixels who are within $n $times the uncertainty above zero

In [22]:
if t is not None :
    t.tv(dark)
dark2=ared.mkdark(range(110,120),clip=3)
if t is not None :
    t.tv(dark2)

  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0110.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0111.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0112.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0113.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/dark120.0114.fits
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/red/UT211031/ARCTIC/da

Here, we'll reduce a dark with and without the combined dark and display them.

In [23]:
if t is no None :
    t.tv(dark,min=-10,max=20)
    t.tv(ared.reduce(110),min=-10,max=20)
    t.tv(ared.reduce(110,dark=dark),min=-10,max=20)

SyntaxError: invalid syntax (4207104084.py, line 1)

<h3>TMO flats

Here's an example of making flat fields for TMO

In [24]:
tred=imred.Reducer('TM61',dir='/home/tmo/20211020')
for filt in (['SG','SR','SI','SZ','SU','Halpha']) :
  flats=glob.glob(tred.dir+'/*Dawn_'+filt+'*')
  flat=tred.mkflat(flats,display=t)
  flat.write(filt+'flat.fits',overwrite=True)



INSTRUMENT: TM61   config: 
  will use format:  /home/tmo/20211020/* {:d}.f*.fits*
         gain:  [0.8]    rn: [ 10.]
         scale:  0.44   
  Biastype : 0
  Bias box: 
    SC    NC    SR    NR
   100  4601  3195    11 
  Trim box: 
    SC    NC    SR    NR
     0  4800     0  3194 
  Norm box: 
    SC    NC    SR    NR
  1200   801  2000   801 
  Reading file: /home/tmo/20211020/Flat_20211020_124812_Dawn_SG_2_003_T1.fts
  subtracting overscan:  509.238
  Reading file: /home/tmo/20211020/Flat_20211020_124751_Dawn_SG_2_002_T1.fts
  subtracting overscan:  509.171
  Reading file: /home/tmo/20211020/Flat_20211020_123408_Dawn_SG_2_001_T1.fts
  subtracting overscan:  508.502
  combining data with median....
  calculating uncertainty....
appending uncertainty
appending bitmask
  Reading file: /home/tmo/20211020/Flat_20211020_125246_Dawn_SR_2_003_T1.fts
  subtracting overscan:  508.761
  Reading file: /home/tmo/20211020/Flat_20211020_125327_Dawn_SR_2_005_T1.fts
  subtracting overscan:  509.